In [1]:
# LinearRegression.py
import numpy as np
from scipy import stats

class LinearRegression: 
    def __init__(self, X, Y, features, response):
        self._X = X 
        self.Y = Y 
        self.features = features
        self.response = response  
 
    @property 
    def X(self):
        X = self._X
        X = np.column_stack([np.ones(X.shape[0]), X])
        return X
    
    @property
    def n(self):
        return self._X.shape[0]
    
    @property
    def d(self):
        return self._X.shape[1] 
     
    @property 
    def b(self):
        X = self.X 
        Y = self.Y
        return np.linalg.pinv( X.T @ X) @ X.T @ Y
        
    # SSE | Residual Sum of Squares | Sum of Squared Error
    # The closer this is to zero the more accuret the prediction (in theory)
    @property 
    def SSE(self):
        X, Y, b = self.X, self.Y, self.b
        return  sum( np.square( Y - X @ b ) )
    
    # SSR | Explained Sum of Squares | Regression Sum of Squares
    @property
    def SSR(self):
        X, Y_mean, b = self.X, self.Y.mean(), self.b
        return sum( np.square( X @ b - Y_mean ) )
    
    # SST | Syy | Total Sum of Squares
    @property 
    def SST(self):
        Y = self.Y
        Y_mean = Y.mean() 
        return sum( np.square(Y-Y_mean) )
    
    # Picks the confidence level
    # If it's below 0.68 it sets it to R2 
    @property 
    def confidence_level(self):
        R2 = self.R2()  
        confidence_levels = [0.997, 0.95, 0.9, 0.8, 0.68, R2]
        return [ cl for cl in confidence_levels if cl <= R2 ][0]

    def print_all(self):
        all = f"""
G
1.     Features: {self.d}
2.  Sample Size: {self.n}
3.     Variance: {self.var()[0]:.5f}
4.  S.Deviation: {self.std()[0]:.5f}
5. Significance: {self.sig()[0]}
6.    Relevance: {self.R2()[0]:.5f}

VG
1. Individual Significance
{ str().join( f"{row}\n{str()}" for row in self.sig_var().split("\n") ) }2. Pairs of Pearsons
{ str().join( f"{row}\n{str()}" for row in self.pearson().split("\n") ) }3. Confidence Interval 
{ str().join( f"{row}\n{str()}" for row in self.con_int().split("\n") ) }4. Confidence Level: {self.confidence_level}
         """
        print(all)

    # The Method that Calculates The Variance | S^ | sigma^2 
    # On average how much our predicted responses will diviate from the regression line
    def var(self):
        SSE, n, d = self.SSE, self.n, self.d 
        return SSE/(n - d - 1)

    # The Method that Calculated The Standard Deviation | S | sigma
    # Measure the same thing as the Variances but in a more readable but less fair unit
    def std(self):
        var = self.var()
        return np.sqrt(var)
    
    # The Method that reports The Significance of the Regression
    # The closer the pvalue is to zero the less likely it is that the correlation
    # we observe between the features and the respons is coincidental. 
    # We want the p-value to be less than 0.05 aka 5% to confidently reject the H0 hypothesis
    def sig(self): 
        SSR, d, n, var = self.SSR, self.d, self.n, self.var()
        
        sig_statistic = (SSR/d)/var
     
        # Survival Function of the F-Distrubution
        p_significance = stats.f.sf(sig_statistic, d, n-d-1)
        return p_significance
    
    # The method that reports The Relevance of Regression
    # Reports how big of a range our model can reliably predict. 
    # So if our R2 value is 0.90 than we could predict 90% of 
    # all responses within, or relatively close to, the standard deviation
    def R2(self):
        SSR, SST = self.SSR, self.SST
        return SSR/SST


    # Significance of the Variables
    # This reports the significance each feature have on the model
    def sig_var(self):   
        X, b, d, n, std, var, features = self.X, self.b, self.d, self.n, self.std(), self.var(), self.features
    
        # Variance/Covariance Matrix
        c = np.linalg.pinv( (X.T @ X) )*var

        # Significans Statisitca Array
        ssa = [ b[i]/(std * np.sqrt(c[i,i])) for i in range(1, c.shape[1])]
        
        cdf = stats.t.cdf(ssa, n-d-1)
        sf =  stats.t.sf(ssa, n-d-1)
        p = [ 2 * min(cdf[idx], sf[idx]) for idx in range(len(ssa)) ]

        result = str().join( f"{features[idx]:<10}: pvalue = {p[idx][0]}\n" for idx in range(len(p))  )
        return result

    # The Method that calculates the Pearson number between all pairs of parameters
    # Reports how muc correlation exists between each pair of features
    # the closer these values are to zero the less correlation exists between the pair
    def pearson(self):
        X, features = self.X, self.features
        
        result = list()
        
        X = X[:,1:]
        for idx in range(len(features)):
            for idy in range(idx):
                if idy == idx:
                    continue 
                p = stats.pearsonr(X.T[idx], X.T[idy])    
                result.append(f"{features[idx]:<9} VS {features[idy]:<9} : {p[0]:.10f}\n")
        
        return str().join(result[::-1])
    
    # The method that calculates the Confidence Interval
    def con_int(self):
        X,  b, n, d, var, std, features = self.X, self.b, self.n, self.d, self.var(), self.std(), self.features
      
        a = 1-self.confidence_level 
        df = n-d-1
        results = list()

        # Variance/Covariance Matrix
        c = np.linalg.pinv( (X.T @ X) )*var
        features.insert(0, "bias")
        for i in range(d+1):           
            ci = (b[i], stats.t.ppf(a/2, df) * std * np.sqrt(c[i][i]))

            # Returns the result in the order of low to high
            low_to_high = min((ci[0][0]-ci[1][0]), (ci[0][0]+ci[1][0])),max((ci[0][0]-ci[1][0]),(ci[0][0]+ci[1][0]))
            result = f"{features[i-1]}: {ci[0][0]:.5f} ± {abs(ci[1][0]):.5f} | interval:[{low_to_high[0]:.5f} <> {low_to_high[1]:.5f}]\n"
            results.append(result)
        
        return str().join(results)   
    
 

In [2]:
import pandas as pd
path = "../Resources/" 

In [3]:
# An instance without the Observer
data_set = pd.read_csv(path+"Small-diameter-flow.csv") 
X = data_set.drop(columns=["Unnamed: 0", "Observer", "Flow"])
Y = data_set[["Flow"]]
flow = LinearRegression(X.values, Y.values, list(X.columns), Y.columns[0])

flow.print_all()



G
1.     Features: 3
2.  Sample Size: 198
3.     Variance: 0.00631
4.  S.Diviation: 0.07943
5. Significance: 3.8879169683951874e-246
6.    Relevance: 0.99712

VG
1. Individual Significance
Kinematic : pvalue = 2.2799778946075336e-236
Geometric : pvalue = 0.0
Inertial  : pvalue = 1.9192831125684836e-242

2. Pairs of Pearsons
Inertial  VS Geometric : 0.9183300309
Inertial  VS Kinematic : 0.9686707505
Geometric VS Kinematic : 0.8631350761

3. Confidence Interval 
Inertial: -2.55979 ± 0.10089 | interval:[-2.66069 <> -2.45890]
bias: 0.86872 ± 0.01163 | interval:[0.85709 <> 0.88034]
Kinematic: 3.61042 ± 0.00785 | interval:[3.60257 <> 3.61827]
Geometric: -0.75369 ± 0.00938 | interval:[-0.76307 <> -0.74430]

4. Confidence Level: 0.997
         


In [4]:
# An instance with the Observer
data_set = pd.read_csv(path+"Small-diameter-flow.csv") 
X = data_set.drop(columns=["Unnamed: 0", "Flow"])
Y = data_set[["Flow"]]
flow_obs = LinearRegression(X.values, Y.values, list(X.columns), Y.columns[0])

#flow_obs.print_all()
print(flow_obs.sig_var())


Kinematic : pvalue = 5.730580151466907e-236
Geometric : pvalue = 0.0
Inertial  : pvalue = 1.1628066959545507e-241
Observer  : pvalue = 2.3422411107265474e-44



### 4. Discussion
Question: ”Is there an observer bias in the data collected for the small-diameter flow measurements?"

Answer: For there to be an Observer Bias the categorical Observer feature\
needs to be significant.\
To prove significance we need to reject the H0 hypothesis.\
To confidently reject the H0 hypothesis we need to see a P-Value off less\
then 0.05 meaning that there would be less then 5% chance that the\
correlation between the Observer and Flow would be coincidental.

We can check the p-value between the Observer and Flow using the\
sig_var method, as was done in the cell above.\
This gives us a p-value for the Observer of 2.3422411107265474e-44,\
which is way less then 0.05 which means that we can, with confidence,\
reject the H0 hypothesis, the Observer feature is significant, the\
correlation is not accidental and therefore there is most likely an Obserber Bias. 